This notebook acquires and processes Harmonized Landsat/Sentinel (HLS) and Land Surface Temperature (LST) data for input into a predictive model of urban tree canopy. 

In [ ]:
from pathlib import Path

from osgeo import gdal
import earthaccess

from src.data_download import Site, DataDownloader, DataProcessor



First, you need to set up and authenticate an earthaccess account to access data from NASA's EarthData API.  
Instructions on how to do this can be found here: https://earthaccess.readthedocs.io/en/stable/quick-start/

In [ ]:
# authenticate account
earthaccess.login(strategy='netrc')

# set root directory to current working directory
root = Path.cwd()

# set GDAL configurations used to access LP DAAC Cloud Assets via vsicurl 
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
#gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')
gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL', 'YES')
gdal.SetConfigOption('GDAL_HTTP_MAX_RETRY', '10')
gdal.SetConfigOption('GDAL_HTTP_RETRY_DELAY', '0.5')

gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','.tif, .tiff')

### Downloading Data

Initialize a `Site` object for the date of interest. When you do so, a folder at /data/site_name will be created to house the downloaded satellite data. Make sure that 'nyc_boundary.gpkg' is saved to the data folder.

In this case, we will create a site object for the year 2023 and called it nyc23.

In [ ]:
nyc23 = Site(root=root,site_name='nyc23',epsg=26918)

To create our training data, we acquire data from three consecutive years and composite them together. Create a `DataDowloader` object for every year that you want to acquire data from. Pass in the site object and the desired year. 

In [ ]:
dl_1 = DataDownloader(nyc23,year='2022')
dl_2 = DataDownloader(nyc23,year='2023')
dl_3 = DataDownloader(nyc23,year='2024')

Calling `download_reflectance_data()` queries NASA's earthdata API and identifies suitable cloud-free images, then masks, scales, and saves all data.

In [ ]:
dl_1.download_reflectance_data()
dl_2.download_reflectance_data()
dl_3.download_reflectance_data()

Calling `downlaod_lst_data()` queries Microsoft's Planetary Computer and acquires, masks, scales and saves all suitable LST imagery.

In [ ]:
dl_1.download_lst_data()
dl_2.download_lst_data()
dl_3.download_lst_data()



The satellite imagery is saved to data/site_name as:  
sentinel_data_{year}.nc   
landsat_data_{year}.nc   
lst_data_{year}.nc  

A text file containing the dates of the satellite imagery acquisitions is also saved to data/site_name

### Processing Data

Tp begin processing the data you just downloaded, initialize a `DataProcessor` object. Pass in the site object and the target year. The data from all the years associated with this site object are collapsed into a single time series. 

In [ ]:
p = DataProcessor(nyc23,2023)

Imagery acquired at different times can 'jitter' and cause feature misalignment issues, especially when compositing data.   
More details here: https://medium.com/sentinel-hub/how-to-co-register-temporal-stacks-of-satellite-images-5167713b3e0b

Calling `coregister_bands()` aligns the bands of each satellite to the median red band.
  
Next, `combine_bands()` creates median monthly composites of all band data from both satellites. The option to add tree canopy (`add_tc`) is set to False because no ground truth is available for this year. 

`combine_indices()` calculates a series of spectral indices for each satellite, and then creates median monthly composites.

The same steps are then applied to the LST data with `combine_lst()` and `coregister_bands()`

Processed data are saved to data/site_name/ as:  
{site_name}_hls_bands.nc  
{site_name}_hls_indices.nc  
{site_name}_lst.nc  

In [ ]:
p.coregister_bands(satellite='sentinel',lst=False)
p.coregister_bands(satellite='landsat',lst=False)


p.combine_bands(add_tc=False)
p.combine_indices()


p.combine_lst()
p.coregister_bands(satellite='none', lst=True)

### Processing Data when Lidar-derived ground truth data is available

When ground truth data is available, the same process is used with a few alterations. The lidar tree canopy raster should be saved to data/site_name as {site_name}_tree_canopy.tif 

Initialize a `Site` object and coregister the bands as before. Then, call `align_to_tree_canopy()` to ensure he self-aligned bands are also aligned to the tree canopy raster. 

In [ ]:
nyc17 = Site(root=root,site_name='nyc17',epsg=26918)

p = DataProcessor(nyc17,2017)

#coregister HLS bands 
p.coregister_bands(satellite='sentinel',lst=False)
p.coregister_bands(satellite='landsat',lst=False)


p.align_to_tree_canopy(satellite='sentinel')
p.align_to_tree_canopy(satellite='landsat')

# create median monthly composites of bands and indices
p.combine_bands()
p.combine_indices()

# composite and align lst data
p.combine_lst()
p.coregister_bands(satellite='none', lst=True)
p.align_to_tree_canopy(satellite='none',lst=True) ## align to the tree canopy raster

# this output shows the alignment warp matrix - values should typically be between -1 - 1

[[ 1.          0.         -0.41707057]
 [ 0.          1.         -0.32940337]]
[[ 1.          0.         -0.46982217]
 [ 0.          1.         -0.3209806 ]]
Aggregating Landsat-only bands...
Aggregating Sentinel-only bands...
Combining results...
calculating shared bands
Combining results...
[[1.         0.         0.11735997]
 [0.         1.         0.00302923]]
